In [1]:
import spacy 
import pyArango
import os
from os import path
import time
import glob
import pandas as pd
from arango import ArangoClient

Initialisation de la connection avec la base de données 

In [2]:
# Initialize the client for ArangoDB.
client = ArangoClient(hosts="http://localhost:8529")

In [3]:
sys_db = client.db('_system', username='root',password='root')

si besoin de repartir à zero:

In [4]:
sys_db.delete_database('text')

True

Check si la base de données existe déjà et création si besoin.

In [5]:
if not sys_db.has_database('text'):
    
    sys_db.create_database('text')
    db = client.db("text", username="root", password="root")
    graph = db.create_graph('text_explorer')
    # création des collections
    docs = graph.create_vertex_collection('docs')
    sentences = graph.create_vertex_collection('sentences')
    tokens = graph.create_vertex_collection('tokens')
    lemmas = graph.create_vertex_collection('lemmas')
    # création des arrêtes
    is_from = graph.create_edge_definition(
        edge_collection='is_from',
        from_vertex_collections=['sentences'],
        to_vertex_collections=['docs']
    )
    contracts_to = graph.create_edge_definition(
        edge_collection='contracts_to',
        from_vertex_collections=['tokens'],
        to_vertex_collections=['lemmas']
    )
    syntagmatic_link = graph.create_edge_definition(
        edge_collection='syntagmatic_link',
        from_vertex_collections=['tokens'],
        to_vertex_collections=['tokens']
    )
else:
    db = client.db("text", username="root", password="root")

L'objectif maintenant est de remplir ces champs avec les données extraites depuis le texte

Récupération des documents

In [6]:
def get_text(path):
    with open(path, encoding='utf8') as f:
        return(f.read().replace('\n',' '))
        f.close()

In [7]:
dir_path = os.getcwd()
dir_path

'/home/paul/projects/text explorer'

Liste des fichiers présents dans le dossier choisi pour l'analyse

In [8]:
files = glob.glob('/home/paul/projects/text_for_app/*.{}'.format('txt'))
files

['/home/paul/projects/text_for_app/jean_blog.txt',
 '/home/paul/projects/text_for_app/emploi étudiant et inégalités sociales.txt']

In [9]:
def get_filename_from_path(path):
    return os.path.normpath(path).split(os.sep)[-1]

In [10]:
documents = pd.DataFrame({'filepath':files,
                          'doc_name':[get_filename_from_path(filepath) for filepath in files],
                          'doc_number':list(range(0,len(files)))})
documents

,filepath,doc_name,doc_number
0,/home/paul/projects/text_for_app/jean_blog.txt,jean_blog.txt,0
1,/home/paul/projects/text_for_app/emploi étudia...,emploi étudiant et inégalités sociales.txt,1


In [11]:
def insert_docs(documents_found):
    in_db_doclist = pd.DataFrame(list(db.aql.execute('''FOR doc in docs RETURN doc''')))
    if in_db_doclist.shape == (0,0):
        pass
    else :
        documents_found = documents_found[~documents_found['doc_name'].isin(in_db_doclist['doc_name'])]
        last_doc_number = in_db_doclist['doc_number'].max()
        documents_found['doc_number'] = documents_found['doc_number'] + last_doc_number + 1
    
    dict_list_documents_to_insert = []
    for doc_name, number, path  in zip(documents_found['doc_name'], documents_found['doc_number'], documents_found['filepath']):
        dict_list_documents_to_insert.append({'_key':f'doc{number}',
                                                'doc_name':doc_name,
                                                'doc_path':path,
                                                'doc_number':number,
                                                'processed':'False'})
    db.collection('docs').import_bulk(dict_list_documents_to_insert)

In [12]:
insert_docs(documents)

# Chargement du modèle

In [13]:
nlp = spacy.load('fr_core_news_lg')

Traitement des fichiers par le modèle

## se renseigner sur comment faire du traitement en batch
    - mesurer l'empreinte mémoire de l'opération et ajuster combien de documents en même temps peuvent être traités

In [14]:
texts_to_process_df = pd.DataFrame(
                        list(
                            db.aql.execute('''
                                            FOR doc in docs
                                            FILTER doc.processed == 'False'
                                            RETURN {path :doc.doc_path, number : doc.doc_number}
                                            ''')
                            )
                        )
texts_to_process_df

,path,number
0,/home/paul/projects/text_for_app/jean_blog.txt,0
1,/home/paul/projects/text_for_app/emploi étudia...,1


Voir l'empreinte mémoire de l'opération. Découper l'output des textes à traiter pour ne pas tout fournir direction à l'opération ci dessous

In [15]:
processed_docs = list(nlp.pipe([get_text(path) for path in texts_to_process_df['path']]))

In [16]:
def insert_sentences(processed_doc, doc_number):
    dict_sentences_to_insert = []
    for sentence_number, sentence in enumerate(processed_doc.sents):
        if sentence.text != ' ':
            dict_sentences_to_insert.append({'_key':f'doc{doc_number}sent{sentence_number}',
                                             'content':sentence.text})
        else:
            pass
    db.collection('sentences').import_bulk(dict_sentences_to_insert)

In [17]:
for processed_text, doc_number in zip(processed_docs,texts_to_process_df['number']):
    insert_sentences(processed_text,doc_number)

Insertion des tokens et lemmas

Récupérer le vocabulaire token et lemma de chaque doc et les insérer dans la db

- clé = nombre arbitraire d'insertion
- valeur = le token ou lemma

Pour insérer de nouveaux mots par la suite faire une requette sur la table des tokens / lemma et insérer ceux qui ne sont pas déjà présents 

pour les relations faire une requete par phrase où l'on récupère chaque mot et sa clé associée
- relier ensuite les mots entre eux en utilisant les clés et la modalité de cette relation

Comment relier les phrases aux documents et les mots aux phrases ? 

### Extraction des correspondances lemmes / tokens depuis le texte

In [18]:
def get_vocab_table_from_text(processed_text):
    tokens, lemmas = [], []
    for token in processed_text:
        if not token.is_punct and not token.is_stop and not token.is_space and not token.is_digit:
            tokens.append(token.text.lower())
            lemmas.append(token.lemma_.lower())
    vocab_table = pd.DataFrame({'token':tokens,
                                'lemma':lemmas})
    return vocab_table

### Construction de deux tables contenant les occurences uniques des lemmes et tokens

In [19]:
def unique_vocabulary(token_lemma_table,word_type):
    return(token_lemma_table[word_type].drop_duplicates().reset_index(drop=True))

### Extraction du vocabulaire existant depuis la base de données

In [20]:
def get_vocab_in_db():
    vocab_tokens_from_db = pd.DataFrame(
        list(db.aql.execute("""
            FOR doc in tokens
            RETURN {word :doc.token, key : doc._key}
            """))
        )
        
    vocab_lemmas_from_db = pd.DataFrame(
        list(db.aql.execute("""
            FOR doc in lemmas
            RETURN {word :doc.lemma, key : doc._key}
            """))
        )
    return vocab_tokens_from_db, vocab_lemmas_from_db

### Comparaison du vocabulaire extrait du texte avec celui déjà contenu dans la db
- On ne rajoute que les instances jamais vues

In [21]:
def keep_only_new_vocab(vocab_from_text,vocab_from_db):
    new_vocab = vocab_from_text[~vocab_from_text.isin(vocab_from_db['word'])].reset_index(drop=True)
    return new_vocab

A refactoriser :

In [28]:
def add_vocab_to_db(processed_text):
    token_to_lemma_table = get_vocab_table_from_text(processed_text)
    
    text_tokens_series = unique_vocabulary(token_to_lemma_table,'token')
    text_lemmas_series = unique_vocabulary(token_to_lemma_table,'lemma')
    
    tokens_from_db, lemmas_from_db = get_vocab_in_db()
    
    if tokens_from_db.shape == (0,0):
        dict_list_tokens_to_insert = []
        for index, token in zip(text_tokens_series.index,text_tokens_series.values):
            dict_list_tokens_to_insert.append({"_key":f'token{index}',
                                               'token':token})
        db.collection('tokens').import_bulk(dict_list_tokens_to_insert)

        dict_list_lemmas_to_insert = []
        for index, lemma in zip(text_lemmas_series.index,text_lemmas_series.values):
            dict_list_lemmas_to_insert.append({"_key":f'lemma{index}',
                                               'lemma':lemma})
        db.collection('lemmas').import_bulk(dict_list_lemmas_to_insert)

    else :
        new_vocab_tokens = keep_only_new_vocab(text_tokens_series,tokens_from_db)
        new_vocab_lemmas = keep_only_new_vocab(text_lemmas_series,lemmas_from_db)

        new_vocab_tokens.index = new_vocab_tokens.index + tokens_from_db.shape[0] + 1
        new_vocab_lemmas.index = new_vocab_lemmas.index + lemmas_from_db.shape[0] + 1

        dict_list_tokens_to_insert = []
        for index, token in zip(new_vocab_tokens.index,new_vocab_tokens.values):
            dict_list_tokens_to_insert.append({"_key":f'token{index}',
                                               'token':token})
        db.collection('tokens').import_bulk(dict_list_tokens_to_insert)

        dict_list_lemmas_to_insert = []
        for index, lemma in zip(new_vocab_lemmas.index,new_vocab_lemmas.values):
            dict_list_lemmas_to_insert.append({"_key":f'lemma{index}',
                                               'lemma':lemma})
        db.collection('lemmas').import_bulk(dict_list_lemmas_to_insert)

In [29]:
add_vocab_to_db(processed_docs[0])

In [30]:
add_vocab_to_db(processed_docs[1])

# todo :
- insérer les relations

###

Extraction d'une forme tabulaire de la structure syntagmatique.
  > Le processus est itéré sur l'ensemble des phrases du document

on obtient une liste de dataframes contenant la structure syntagmatique pour chaque phrase

In [18]:
def create_dependancy_df_list(processed_text):
    df_list = []
    for sentence in processed_text.sents:
        token_text, token_dep, token_head_text, token_head_pos = [], [], [], []
        for token in sentence:
            if not token.is_punct and not token.is_stop and not token.is_space:
                token_text.append(token.text)
                token_dep.append(token.dep_), 
                token_head_text.append(token.head.text), 
                token_head_pos.append( token.head.pos_)
        df = pd.DataFrame({'token':token_text,
                           'dep':token_dep,
                           'head_text':token_head_text,
                           'head_pos':token_head_pos})    
        if not df.empty:
            df_list.append(df)
        else:
            pass
    return df_list

In [19]:
file_1_processed = nlp(get_text(documents['filepath'][0]))
file_2_processed = nlp(get_text(documents['filepath'][1]))

In [20]:
create_dependancy_df_list(file_1_processed)[0]

,token,dep,head_text,head_pos
0,faire,xcomp,vais,VERB
1,inutilement,advmod,faire,VERB
2,durer,xcomp,faire,VERB
3,suspense,obj,durer,VERB
4,réponse,nsubj,oui,ADV
5,oui,parataxis,vais,VERB
